<a href="https://colab.research.google.com/github/carsofferrei/04_data_processing/blob/main/spark/challenges/challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHALLENGE 1
##  Implement INGESTION process
- Set up path in the "lake"
  - !mkdir -p /content/lake/bronze

- Read data from API https://api.carrismetropolitana.pt/
  - Endpoints:
    - vehicles
    - lines
    - municipalities
  - Use StructFields to enforce schema

- Transformations
  - vehicles
    - create "date" extracted from "timestamp" column (format: hh24miss)

- Write data as PARQUET into the BRONZE layer (/content/lake/bronze)
  - Partition "vehicles" by "date" column
  - Paths:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities
  - Make sure there is only 1 single parquet created
  - Use overwrite as write mode

# Setting up PySpark

In [109]:
%pip install pyspark

In [1]:
# Import SparkSession

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Carla_Ferreira_API_challenge').config('spark.ui.port', '4050').getOrCreate()
sc = spark.sparkContext

In [2]:
import requests
from pyspark.sql.types import *

def readFromAPI(url: str, schema: StructType = None):
  response = requests.get(url)
  rdd = sc.parallelize(response.json())

  if schema:
    df = spark.read.schema(schema).json(rdd)
  else:
    df = spark.read.json(rdd)
  return df

In [3]:
#Schemas
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)
                             ])


lines_schema = StructType([StructField('_corrupt_record', StringType(), True),
                           StructField('color', StringType(), True),
                           StructField('facilities', StringType(), True),
                           StructField('id', StringType(), True),
                           StructField('localities', ArrayType(StringType()), True),
                           StructField('long_name', StringType(), True),
                           StructField('municipalities', ArrayType(StringType()), True),
                           StructField('patterns', ArrayType(StringType()), True),
                           StructField('routes', StringType(), True),
                           StructField('short_name', StringType(), True),
                           StructField('text_color', StringType(), True)
                           ])

municipalities_schema = StructType([StructField('district_id', StringType(), True),
                                    StructField('district_name', StringType(), True),
                                    StructField('id', StringType(), True),
                                    StructField('name', StringType(), True),
                                    StructField('prefix', StringType(), True),
                                    StructField('region_id', StringType(), True),
                                    StructField('region_name', StringType(), True)
                                    ])

In [4]:
vehicles = readFromAPI("https://api.carrismetropolitana.pt/vehicles", schema = vehicle_schema)
lines = readFromAPI("https://api.carrismetropolitana.pt/lines", schema = lines_schema)
municipalities = readFromAPI("https://api.carrismetropolitana.pt/municipalities", schema = municipalities_schema)

In [5]:
# create "date" extracted from "timestamp" column (format: hh24miss) -- this format doesnt make sense, the correct is yyyyMMdd
from pyspark.sql.functions import date_format
vehicles = vehicles.withColumn("date", date_format('timestamp', "yyyyMMdd"))

vehicles.show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+--------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|    date|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+--------+
|    201|20241123-64020071...| IN_TRANSIT_TO|44|12066|38.529713|   4404|-8.885915|  4404_0_3|  4404_0|            SCHEDULED|112170000007|      0.0| 160067|2024-11-23 09:58:53|4404_0_3|2300|100...|20241123|
|    165|20241123-64020035...| IN_TRANSIT_TO|44|12745|38.663094|   4730|-9.161282|  4730_2_1|  4730_2|            SCHEDULED|113030000007|6.9444447| 140089|2024-11-23 09:58:52|4

In [6]:
from pyspark.sql import functions as F

vehicles.select("date").distinct().agg(F.min("date").alias("min_date")).show()
vehicles.select("date").distinct().agg(F.max("date").alias("max_date")).show()

# there's only one day in this file.

+--------+
|min_date|
+--------+
|20241123|
+--------+

+--------+
|max_date|
+--------+
|20241123|
+--------+



Write data as PARQUET into the BRONZE layer (/content/lake/bronze)

*   Partition "vehicles" by "date" column
*   Paths: /content/lake/bronze/vehicles, /content/lake/bronze/lines
, /content/lake/bronze/municipalities

Make sure there is only 1 single parquet created /
Use overwrite as write mode

Declare the directory to write the data that came from API carrismetropolitana

In [7]:
!mkdir -p /content/lake/bronze

path_vehicles = "/content/lake/bronze/vehicles"
path_lines = "/content/lake/bronze/lines"
path_municipalities = "/content/lake/bronze/municipalities"

In [8]:
(vehicles
.coalesce(1)
.write
.mode("overwrite")
.partitionBy("date")
.format("parquet")
.save(path_vehicles)
)


(lines
.write
.mode("overwrite")
.format("parquet")
.save(path_lines)
)


(municipalities
.write
.mode("overwrite")
.format("parquet")
.save(path_municipalities)
)

# CHALLENGE 2
##  Implement CLEANSING process
- Set up path in the "lake"
  - !mkdir -p /content/lake/silver

- Read data from BRONZE layer as PARQUET:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities

- Transformations
  - vehicles
    - rename "lat" and "lon" to "latitude" and "longitude" respectively
    - remove possible duplicates
    - remove rows when the column CURRENT_STATUS is null
    - remove any corrupted record
  - lines
    - remove duplicates
    - remove rows when the column LONG_NAME is null
    - remove any corrupted record
  - municipalities
    - remove duplicates
    - remove rows when the columns NAME or DISTRICT_NAME are null
    - remove any corrupted record

- Write data as PARQUET into the SILVER layer (/content/lake/silver)
  - Partition "vehicles" by "date"(created in the ingestion)
  - Paths:
    - vehicles - path: /content/lake/silver/vehicles
    - lines - path: /content/lake/silver/lines
    - municipalities - path: /content/lake/silver/municipalities

In [9]:
vehicles = spark.read.parquet("/content/lake/bronze/vehicles", schema = vehicle_schema)

In [10]:
lines = spark.read.parquet("/content/lake/bronze/lines", schema = lines_schema)

In [11]:
municipalities = spark.read.parquet("/content/lake/bronze/municipalities", schema = municipalities_schema)

In [12]:
from pyspark.sql.functions import *

# Vehicles
# rename "lat" and "lon" to "latitude" and "longitude" respectively
vehicles = vehicles.withColumnRenamed("lat", "latitude").withColumnRenamed("lon", "longitude")

# remove possible duplicates
vehicles = vehicles.dropDuplicates()
# remove rows when the column CURRENT_STATUS is null
vehicles = vehicles.filter(vehicles["current_status"].isNotNull())
# remove any corrupted record (????)

In [13]:
from pyspark.sql.functions import *

# Lines
# remove duplicates
lines = lines.dropDuplicates()
# remove rows when the column LONG_NAME is null
lines = lines.filter(lines["long_name"].isNotNull())
# remove any corrupted record
lines = lines.filter(lines["_corrupt_record"].isNull())

In [14]:
# Municipalities
# remove duplicates
municipalities = municipalities.dropDuplicates()
# remove rows when the columns NAME or DISTRICT_NAME are null
municipalities = municipalities.filter(municipalities["name"].isNotNull() & municipalities["district_name"].isNotNull())
# remove any corrupted record (???)

In [15]:
!mkdir -p /content/lake/silver

In [16]:
path_vehicles = "/content/lake/silver/vehicles"
path_lines = "/content/lake/silver/lines"
path_municipalities = "/content/lake/silver/municipalities"

In [17]:
(vehicles
.coalesce(1)
.write
.mode("overwrite")
.partitionBy("date")
.format("parquet")
.save(path_vehicles)
)


(lines
.write
.mode("overwrite")
.format("parquet")
.save(path_lines)
)


(municipalities
.write
.mode("overwrite")
.format("parquet")
.save(path_municipalities)
)

# CHALLENGE 3
##  Implement ENRICH process
- Set up path in the "lake"
  - !mkdir -p /content/lake/gold

- Read data from SILVER layer
  - Paths:
    - vehicles - path: /content/lake/silver/vehicles
    - lines - path: /content/lake/silver/lines
    - municipalities - path: /content/lake/silver/municipalities
  - Use StructFields to enforce schema

- Enrichment
  - Enrich vehicles dataset with information from the line and municipalities
    - join vehicles with lines and municipalities
      - select all columns from vehicles + lines.long_name (name: line_name, format:string) + municipalities.name (name: municipality_name, format: array)
      - Note that "municipalities.name" is an array

- Write data as PARQUET into the GOLD layer (/content/lake/gold)
  - Dataset name: vehicles_enriched
  - Partition "vehicles_enriched" by "date" column
  - Paths:
    - vehicles - path: /content/lake/gold/vehicles_enriched
  - Make sure there is only 1 single parquet created
  - Use overwrite as write mode

In [18]:
vehicles = spark.read.parquet("/content/lake/silver/vehicles", schema = vehicle_schema)

In [19]:
lines = spark.read.parquet("/content/lake/silver/lines", schema = lines_schema)

In [20]:
municipalities = spark.read.parquet("/content/lake/silver/municipalities", schema = municipalities_schema)

In [21]:
# Extract info from the array in municiples
from pyspark.sql.functions import explode
lines_treated = lines.select(
    "facilities"
    , "id"
    , "localities"
    , explode(lines.municipalities).alias("municipalities_id")
).drop_duplicates()

In [22]:
vehicles_lines = vehicles.join(lines_treated, vehicles['line_id'] == lines_treated['id'], how = 'left')
vehicles_enriched = vehicles_lines.join(municipalities, lines_treated['municipalities_id'] == municipalities['id'], how = 'left')
vehicles_enriched.dropDuplicates().show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+--------+----------+----+--------------------+-----------------+-----------+-------------+----+--------+------+---------+-----------+
|bearing|            block_id|current_status|      id| latitude|line_id|longitude|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|    date|facilities|  id|          localities|municipalities_id|district_id|district_name|  id|    name|prefix|region_id|region_name|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+--------+----------+----+--------------------+-----------------+-----------+-------------+----+--------+------+---------+-----------+
|    316| 

In [23]:
vehicles_enriched = vehicles_enriched.select(
      "line_id"
    , "current_status"
    , "schedule_relationship"
    , "shift_id"
    , "speed"
    , "stop_id"
    , "date"
    , "facilities"
    , "municipalities_id"
    , "district_name"
    , "name"
    , "prefix"
    , "region_id"
    , "region_name"
).drop_duplicates()

In [24]:
!mkdir -p /content/lake/gold

In [25]:
path_vehicles_enriched = "/content/lake/gold/vehicles_enriched"

In [26]:
vehicles_enriched.show()

+-------+--------------+---------------------+----------------+---------+-------+--------+----------+-----------------+-------------+-------+------+---------+-----------+
|line_id|current_status|schedule_relationship|        shift_id|    speed|stop_id|    date|facilities|municipalities_id|district_name|   name|prefix|region_id|region_name|
+-------+--------------+---------------------+----------------+---------+-------+--------+----------+-----------------+-------------+-------+------+---------+-----------+
|   1721|    STOPPED_AT|            SCHEDULED|            2609|      0.0| 060055|20241123|        []|             1106|       Lisboa| Lisboa|    06|    PT170|        AML|
|   4701| IN_TRANSIT_TO|            SCHEDULED|    123070000007|      0.0| 090018|20241123|        []|             1106|       Lisboa| Lisboa|    06|    PT170|        AML|
|   2790|    STOPPED_AT|            SCHEDULED|            4018|      0.0| 182258|20241123|        []|             1107|       Lisboa| Loures|    

In [27]:
(vehicles_enriched
.coalesce(1)
.write
.mode("overwrite")
.partitionBy("date")
.format("parquet")
.save(path_vehicles_enriched)
)

# CHALLENGE 4
##  Analyze data

- Query table "vehicles_enriched" in gold layer
- Aggregate data by municipality_name (array)
- Calculate:
  - count of vehicles (id) that pass through that municipality
  - sum speed of vehicles

Questions:
  - What are the top 3 municipalities by vehicles routes?
  - What are the top 3 municipalities with higher vehicle speed on average?


Tips:
- explode array into rows -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html


In [28]:
vehicles_enriched = spark.read.parquet("/content/lake/gold/vehicles_enriched")

In [29]:
vehicles_enriched.filter(vehicles_enriched.name == "Lisboa").show()

+-------+--------------+---------------------+------------+---------+-------+----------+-----------------+-------------+------+------+---------+-----------+--------+
|line_id|current_status|schedule_relationship|    shift_id|    speed|stop_id|facilities|municipalities_id|district_name|  name|prefix|region_id|region_name|    date|
+-------+--------------+---------------------+------------+---------+-------+----------+-----------------+-------------+------+------+---------+-----------+--------+
|   1721|    STOPPED_AT|            SCHEDULED|        2609|      0.0| 060055|        []|             1106|       Lisboa|Lisboa|    06|    PT170|        AML|20241123|
|   4701| IN_TRANSIT_TO|            SCHEDULED|123070000007|      0.0| 090018|        []|             1106|       Lisboa|Lisboa|    06|    PT170|        AML|20241123|
|   4701| IN_TRANSIT_TO|            SCHEDULED|123080000007|3.6111112| 060001|        []|             1106|       Lisboa|Lisboa|    06|    PT170|        AML|20241123|
|   

In [33]:
data = vehicles_enriched.select("line_id", "name","speed").dropDuplicates()

In [34]:
data = vehicles_enriched.groupBy("name").agg(
      count_distinct("line_id").alias("count_line_ids")
    , round(sum("speed"),2).alias("sum_speed")
)

What are the top 3 municipalities by vehicles routes?

**Sintra, Lisboa e Almada**

In [38]:
data.sort(data.count_line_ids.desc()).limit(3).select("name").show()

+------+
|  name|
+------+
|Sintra|
|Lisboa|
|Almada|
+------+



What are the top 3 municipalities with higher speed on average?

**Moita, Barreiro e Montijo**

In [39]:
data.withColumn(
    "average_speed",
    round((col("sum_speed") / col("count_line_ids")),2)
).orderBy(
    col("average_speed").desc()
).limit(3).select("name").show()

+--------+
|    name|
+--------+
|   Moita|
|Barreiro|
| Montijo|
+--------+

